# 1. Load Required Libraries

In [2]:
import pandas as pd
import numpy as np
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
#Get the working directory will be usefull on next steps
path = os.getcwd()

# 2. GMAIL API

In [ ]:
#2.a. Define the scope
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [ ]:
#2.b. Create credentials file if doesn't exist
if os.path.exists('token.pickle'):
    # Read the token from the file and store it in the variable creds
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
creds

In [43]:
#2.c Connect to the Gmail API
service = build('gmail', 'v1', credentials=creds)

#2.d Request a list of all the messages
result = service.users().messages().list(userId='me').execute()

In [44]:
messages = result.get('messages')

In [45]:
#2.e Create a DataFrame with what is needed
df = pd.DataFrame()

# iterate through all the messages
for msg in messages:
    # Get the message from its id
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    
    try:
    # Get value of 'payload' from dictionary 'txt'
        payload = txt['payload']
        headers = payload['headers']

        # Look for Subject and Sender Email in the headers
        for d in headers:
            #subject = headers[['name'] == 'Subject']['value']
            #sender = headers[['name'] == 'From']['value']
            if d['name'] == 'Subject':
                subject = d['value']
            if d['name'] == 'From':
                sender = d['value']
            if d['name'] == 'Date':
                timestamp = d['value']

            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base 64 decoder.
        #parts = payload.get('parts')[0]
        #data = parts['body']['data']
        data = payload['body']['data']
        data = data.replace("-","+").replace("_","/")
        decoded_data = base64.b64decode(data)

        # Now, the data obtained is in lxml. So, we will parse
        # it with BeautifulSoup library
        soup = BeautifulSoup(decoded_data , "lxml")
        body = soup.body()

        #if sender == 'srv812.main-hosting.eu':
        if subject == 'YOUR COMPANY - New Lead': #This is the condition I used because that's how the subject of emails is.
            d = {'timestamp': timestamp,
                'message': body,
                'id' : msg['id']
                        }
            df_aux = pd.DataFrame(data = d, index =[''])
            df = df.append(df_aux)
        
    except:
        pass
    
df = df.reset_index()

df = df.reset_index()

df  

,level_0,index,timestamp,message,id
0,0,,"Wed, 12 Oct 2022 17:08:23 +0000",[Facundo\r\nAlvarinio de Parga\r\ncocoloco@gma...,183cd2b431ee7f25
1,1,,"Wed, 12 Oct 2022 15:54:48 +0000",[F\r\nArmentano\r\na@gmail.com\r\n43\r\nBot de...,183cce7e8ec0b671
2,2,,"Wed, 12 Oct 2022 12:39:42 +0000",[a\r\nb\r\na@g.com\r\n4\r\nBot de Arbitrajes (...,183cc354931b7eea
3,3,,"Wed, 12 Oct 2022 12:38:57 +0000",[ad\r\nd@m.com\r\n3\r\nBot de Arbitrajes (Ingr...,183cc349460b345b


In [4]:
#2.f Clean & Prepare Data

In [46]:
df['message'] = df['message'].astype(str)

In [47]:
df_1 = df['message'].str.split(r'(\r\n)',expand=True)
df_1[0] = df_1[0].str.replace('<p>', '')
df_1 = df_1[[0,2,4,6,8,10]]


In [48]:
df_1 = df_1[[0,2,4,6,8,10]]
df_1.columns = ['nombre','apellido','mail','telefono','producto','comment']
df_1 = df_1.reset_index()

In [49]:
df = df[['timestamp','id','level_0']]

In [50]:
database = pd.merge(df,df_1,left_on='level_0',right_on='index')
database = database.drop(['level_0','index'],axis = 1)
database

,timestamp,id,nombre,apellido,mail,telefono,producto,comment
0,"Wed, 12 Oct 2022 17:08:23 +0000",183cd2b431ee7f25,Facundo,Alvarinio de Parga,cocoloco@gmail.com,+5492613359628,Bot de Arbitrajes (Ingresos Activos),"Hola, muy bueno todo!"
1,"Wed, 12 Oct 2022 15:54:48 +0000",183cce7e8ec0b671,F,Armentano,a@gmail.com,43,Bot de Arbitrajes (Ingresos Activos),HOl!
2,"Wed, 12 Oct 2022 12:39:42 +0000",183cc354931b7eea,a,b,a@g.com,4,Bot de Arbitrajes (Ingresos Activos),
3,"Wed, 12 Oct 2022 12:38:57 +0000",183cc349460b345b,ad,d@m.com,3,Bot de Arbitrajes (Ingresos Activos),,</p>


In [51]:
database['timestamp'] = pd.to_datetime(database['timestamp'])
database['id'] = database['id'].astype(str)
database['nombre'] = database['nombre'].astype(str)
database['apellido'] = database['apellido'].astype(str)
database['apellido'] = database['apellido'].astype(str)
database['telefono'] = database['telefono'].astype(str)
database['producto'] = database['producto'].astype(str)
database['comment'] = database['comment'].astype(str)
database.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   timestamp  4 non-null      datetime64[ns, UTC]
 1   id         4 non-null      object             
 2   nombre     4 non-null      object             
 3   apellido   4 non-null      object             
 4   mail       4 non-null      object             
 5   telefono   4 non-null      object             
 6   producto   4 non-null      object             
 7   comment    4 non-null      object             
dtypes: datetime64[ns, UTC](1), object(7)
memory usage: 288.0+ bytes


In [52]:
database = database[database['timestamp'] > '2022-10-12 17:00:00']

# 3. Google Sheets and Google Drive APIs

In [53]:
#3.a Define scopes, present credentials and select sheet and worksheet. 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
#path = os.getcwd()
creds = ServiceAccountCredentials.from_json_keyfile_name(path+'/credentials_sheets.json', scope)
client = gspread.authorize(creds)
sheet = client.open('your_google_sheets') #Name of your google sheets --> You have to give access to the sheet in the console!
sheet_instance = sheet.get_worksheet(0) #First Sheet
records_data = sheet_instance.get_all_records()
records_data

[{'timestamp': '2022-10-12 17:08:23+00:00',
  'id': '183cd2b431ee7f25',
  'nombre': 'Facundo',
  'apellido': 'Alvarinio de Parga',
  'mail': 'cocoloco@gmail.com',
  'telefono': 5492613359628,
  'producto': 'Bot de Arbitrajes (Ingresos Activos)',
  'comment': 'Hola, muy bueno todo!',
  'link_whatsapp': 'https://wa.me/+5492613359628'}]

In [54]:
database['timestamp'] = database['timestamp'].astype(str)

/var/folders/dc/4lx8s62j5yv14742tqpdh0fw0000gn/T/ipykernel_63661/2083406783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  database['timestamp'] = database['timestamp'].astype(str)


In [55]:
#3.b Update Google Sheets
sheet_instance.update([database.columns.values.tolist()] + database.values.tolist())

{'spreadsheetId': '14I2HHpJlQ_H43KA8GxM0V1YkaU5pBMejqb31VK2zsuE',
 'updatedRange': 'bdd!A1:H2',
 'updatedRows': 2,
 'updatedColumns': 8,
 'updatedCells': 16}